In [ ]:
from groq import Groq
import os
from dotenv import load_dotenv

# Load environment 
load_dotenv()

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Initialize the conversation with a system message to set the "Persona"
messages = [
    {
        "role": "system",
        "content": "You are a highly skilled Trading Analyst. Use your tools to provide data-backed insights."
    }
]

print("--- AI Trading Analyst Active (Type 'quit' to exit) ---")

while True:
    # 1. Get user input
    user_input = input("\nYou: ")
    
    # Exit condition
    if user_input.lower() in ["quit", "exit", "bye"]:
        print("Analyst: Goodbye and good luck with your trades!")
        break

    # 2. Add user message to the history
    messages.append({"role": "user", "content": user_input})

    # 3. Call the model
    try:
        completion = client.chat.completions.create(
            model="groq/compound",
            messages=messages,
            # Temperature 0 keeps reasoning models consistent
            temperature=0, 
        )

        # 4. Extract and print the response
        assistant_response = completion.choices[0].message.content
        print(f"\nAnalyst: {assistant_response}")

        # 5. Add the assistant's response to history so it has context for the next turn
        messages.append({"role": "assistant", "content": assistant_response})

    except Exception as e:
        print(f"Error: {e}")